In [ ]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from music21 import stream, note, chord, instrument

In [ ]:
# Load training data mappings
with open("D:\Study\Ai_Music_Composer\project\data\generator_training_data.pkl", "rb") as f:
    data = pickle.load(f)

note_to_int = data["note_to_int"]
int_to_note = data["int_to_note"]
mood_to_int = data["mood_to_int"]
int_to_mood = data["int_to_mood"]
SEQ_LEN = data["seq_len"]

n_vocab = len(note_to_int)

In [ ]:
# Load trained generator model
generator = load_model("D:\Study\Ai_Music_Composer\project\models\lstm_mood_classifier.h5")

In [ ]:
# Ask user for mood input
print("Available moods:", list(mood_to_int.keys()))
user_mood = input("Enter the mood you want to generate music for: ").strip().lower()

if user_mood not in mood_to_int:
    raise ValueError(f"Mood '{user_mood}' not found! Available: {list(mood_to_int.keys())}")

mood_int = mood_to_int[user_mood]

In [ ]:
# Pick a random starting sequence
start_seq = np.random.choice(list(note_to_int.values()), SEQ_LEN)
pattern = list(start_seq)

In [ ]:
# Generate music
generated_notes = []
num_generate = 200  # Number of notes to generate

for _ in range(num_generate):
    input_seq = np.reshape(pattern, (1, SEQ_LEN))
    input_mood = np.array([mood_int]).reshape(1, 1)

    prediction = generator.predict([input_seq, input_mood], verbose=0)
    next_note_int = np.argmax(prediction)
    next_note = int_to_note[next_note_int]

    generated_notes.append(next_note)

    # Update pattern
    pattern.append(next_note_int)
    pattern = pattern[-SEQ_LEN:]

In [ ]:
# Convert to MIDI
output_notes = []
for n in generated_notes:
    if ("." in n) or n.isdigit():  # Chord
        notes_in_chord = n.split(".")
        chord_notes = [note.Note(int(n_i)) for n_i in notes_in_chord]
        for n_ in chord_notes:
            n_.storedInstrument = instrument.Piano()
        new_chord = chord.Chord(chord_notes)
        output_notes.append(new_chord)
    else:  # Single note
        new_note = note.Note(n)
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

midi_stream = stream.Stream(output_notes)
midi_stream.write("midi", fp=f"generated_{user_mood}.mid")

print(f"✅ Music generated and saved as generated_{user_mood}.mid")